# DeepSNAP Overview
 a Python library to assist efficient deep learning on graphs.
 * supports flexible graph manipulation, standard pipeline, heterogeneous graphs and simple API.

 1. DeepSNAP is easy to be used for the sophisticated graph manipulations, such as feature computation, pretraining, subgraph extraction etc. during/before the training. 
 2. In most frameworks, standard pipelines for node, edge, link, graph-level tasks under inductive or transductive settings are left to the user to code. In practice, there are additional design choices involved (such as how to split dataset for link prediction). DeepSNAP provides such a standard pipeline that greatly saves repetitive coding efforts, and enables fair comparision for models.
 3. Many real-world graphs are heterogeneous graphs. But packages support for heterogeneous graphs, including data storage and flexible message passing, is lacking. DeepSNAP provides an efficient and flexible heterogeneous graph that supports both the node and edge heterogeneity.

# 1) DeepSNAP Heterogeneous Graph


In DeepSNAP we have three levels of attributes:
* **node level** attributes- including `node_feature` and `node_label`.
* **edge level** attributes including `edge_feature` and `edge_label`.
* **graph level** attributes including `graph_feature` and `graph_label`.


DeepSNAP extends its traditional graph representation to include heterogeneous graphs by including the following graph property features:  
* `node_feature`: The feature of each node (`torch.tensor`)
* `edge_feature`: The feature of each edge (`torch.tensor`)
* `node_label`: The label of each node (`int`)
* `node_type`: The node type of each node (`string`)
* `edge_type`: The edge type of each edge (`string`)

where the key **new** features are `node_type` and `edge_type`, which enables us to perform heterogenous message passing.


## Transform NetworkX object G into a deepsnap.hetero_graph.HeteroGraph
* G- networkx object with the following aattributes:
    * node attributes:
        * node_feature (torch.tensor)
        * node_label (int)
        * node_type (str)
    * edge attributes:
        * edge_feature (torch.tensor), optional
        * edge_labe (int), optional
        * edge_type (str)

In [ ]:
from deepsnap.hetero_graph import HeteroGraph

hete = HeteroGraph(G)

* hete.node_types- list of all node types

* hete.num_nodes()- a dictionary (key=node_type, value=#nodes of corresponding type)
    * hete.num_nodes(\<node_type\>)- #nodes of type \<node_type\>



* hete.node_label- a dictionary of node_type and all corresponding nodes (with same type)
    * key- node_type
    * value- a tensor of all nodes with the corresponding node type (torch.Tensor)

* hete.node_label_index- a dictionary of node_type and all corresponding nodes' indexes
    * key- node_type
    * value- a tensor of all nodes with the corresponding node indexes (torch.Tensor)
    
    (each type has its own indexing)

* heter.num_node_labels()- a dictionary where key=node_tpye, value=#nodes with the corresponding label (int)
    * heter.num_node_features(\<node_label\>)- get #nodes with the corresponding \<node_label\> (int)


* heter.num_node_features()- a dictionary where key=node_tpye, value=#nodes with the corresponding type (int)
    * heter.num_node_features(\<node_type\>)- get #nodes with the corresponding \<node_type\> (int)


In [ ]:
hete.node_types

hete.node_label

hete.node_label_index

In [ ]:
# get all nodes' indexes with node_type <node_type> (tensor) from specific graph
# .tolist() convert tensor to list

n0 = hete._convert_to_graph_index(dataset[k].node_label_index[ <node_type>],  <node_type>).tolist()

## Messages
###  Heterogenous Message Types
* When applying message passing in heterogenous graphs, we seperately apply message passing over each message type.
    *  different message types for the different `node_type` and `edge_type` combinations
* messages are viewed as $(\text{src}, \text{relation}, \text{dest})$ and in DeepSNAP: $\big (node_i^{\text{type}_i}, edge_{i,j}^{\text{type}_{i,j}}, node_j^{\text{type}_j} \big )$

* hete.message_types- list of all message types (message = (src_type, relation, dst_type))
*  hete.edge_type - a dictionary, where
    * key=message type 
    * value= list of edges for the correspoding message (same edge_type)

* hetero_graph.num_edges()- a dictionary (key=message_type, value=#edges of corresponding type)
    * message = (src node_type, relation, dst node_type)
    * hete.num_nodes(\<msg_type\>)- #edges of type \<edge_type\>

In [ ]:
# get list of all message types
hete.message_types

# count #edges for each message type
message_type_edges = [(msg_type, len(edges)) for msg_type, edges in hete.edge_type.items()]

## Message Passing Layer

* Heterogeneous MP layer only computes embeddings for the `dst` nodes of a given `message type`.

# Heterogeneous Graph Node Property Prediction

First let's take a look at the general structure of a heterogeneous GNN layer by working through an example:

Let's assume we have a graph $G$, which contains two node types $a$ and $b$, and three message types $m_1=(a, r_1, a)$, $m_2=(a, r_2, b)$ and $m_3=(a, r_3, b)$. Note: during message passing we view each message as (src, relation, dst), where messages "flow" from src to dst node types. For example, during message passing, updating node type $b$ relies on two different message types $m_2$ and $m_3$.

When applying message passing in heterogenous graphs, we seperately apply message passing over each message type. Therefore, for the graph $G$, a heterogeneous GNN layer contains three seperate Heterogeneous Message Passing layers (`HeteroGNNConv` in this Colab), where each `HeteroGNNConv` layer performs message passing and aggregation with respect to *only one message type*. Since a message type is viewed as (src, relation, dst) and messages "flow" from src to dst, each `HeteroGNNConv` layer only computes embeddings for the *dst* nodes of a given message type. For example, the `HeteroGNNConv` layer for message type $m_2$ outputs updated embedding representations *only* for node's with type b. 

---

An overview of the heterogeneous layer we will create is shown below:

![test](https://drive.google.com/uc?export=view&id=1mkp4OeRrvC4iNFTXSywrmI6Pfl5J__gA)

where we highlight the following notation:

- $H_a^{(l)[m_1]}$ is the intermediate matrix of of node embeddings for node type $a$, generated by the $l$th `HeteroGNNConv` layer for message type $m_1$.
- $H_a^{(l)}$ is the matrix with current embeddings for nodes of type $a$ after the $l$th layer of our Heterogeneous GNN model. Note that these embeddings can rely on one or more intermediate `HeteroGNNConv` layer embeddings(i.e. $H_b^{(l)}$ combines $H_b^{(l)[m_2]}$ and $H_b^{(l)[m_3]}$).

Since each `HeteroGNNConv` is only applied over a single message type, we additionally define a Heterogeneous GNN Wrapper layer (`HeteroGNNWrapperConv`). This wrapper manages and combines the output of each `HeteroGNNConv` layer in order to generate the complete updated node embeddings for each node type in layer $l$ of our model.

 More specifically, the $l^{th}$ `HeteroGNNWrapperConv` layer takes as input the node embeddings computed for each message type and node type (e.g. $H_b^{(l)[m_2]}$ and $H_b^{(l)[m_3]}$) and aggregates across message types with the same $dst$ node type. The resulting output of the $l^{th}$ `HeteroGNNWrapperConv` layer is the updated embedding matrix $H_i^{(l)}$ for each node type i. 

Continuing on our example above, to compute the node embeddings $H_b^{(l)}$ the wrapper layer aggregates output embeddings from the `HeteroGNNConv` layers associated with message types $m_2$ and $m_3$ (i.e. $H_b^{(l)[m_2]}$ and $H_b^{(l)[m_3]}$). 

---

With the `HeteroGNNWrapperConv` module, we can now draw a "simplified" heterogeneous layer structure as follows:

<br/>
<center>
<img src="http://web.stanford.edu/class/cs224w/images/colab4/hetero_conv_1.png"/>
</center>
<br/>

---
**NOTE**: 
As reference, it may be helpful to additionally read through PyG's introduciton to heterogeneous graph representations and buidling heterogeneous GNN models: https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html 

# Train-test split
## GraphDataset
* dataset
* dataset.split(transductive=True, split_ratio)
    * each split return a GraphDataset object that is a list of DeepSNAP graphs

In [ ]:
from deepsnap.dataset import GraphDataset

dataset = GraphDataset([hete], task='node')  # hete- DeepSNAP graph
# Splitting the dataset
# returns datasets of graphs
dataset_train, dataset_val, dataset_test = dataset.split(transductive=True, split_ratio=[0.4, 0.3, 0.3])
# dataset_train[0]- dataset of first graphs
datasets = {'train': dataset_train, 'val': dataset_val, 'test': dataset_test}

# Module

### forward_op(x, module_dict, **kwargs):
* A helper function for the heterogeneous operations. Given a dictionary input x, it will return a dictionary with the same keys and the values applied by the corresponding values of the module_dict with specified parameters. The keys in x are same with the keys in the module_dict.
    * x (Dict[str, Tensor]) – A dictionary that the value of each item is a tensor.

    * module_dict (torch.nn.ModuleDict) – The value of the module_dict will be fed with each value in x.

    * **kwargs (optional) – Parameters that will be passed into each value of the module_dict.

### loss_op(pred, y, index, loss_func)
* A helper function for the heterogeneous loss operations. This function will sum the loss of all node types.
    * pred (Dict[str, Tensor]) – A dictionary of prediction results.
    * y (Dict[str, Tensor]) – A dictionary of labels. The keys should match with the keys in the pred.
    * index (Dict[str, Tensor]) – A dictionary of indicies that the loss will be computed on. Each value should be torch.LongTensor. Notice that y will not be indexed by the index. Here we assume y has been splitted into proper sets.

    * loss_func (callable) – The defined loss function.